In [ ]:
import pandas as pd
import numpy as np

# Import the world-happiness-report csv

country_happiness_df = pd.read_csv("./resources/world-happiness-report.csv")
country_happiness_df.head()

In [ ]:
# View column headers
country_happiness_df.columns

In [ ]:
# Make all columns lowercase
country_happiness_df = country_happiness_df.rename(columns={
    "Country name": "country",
    "Life Ladder": "life_ladder",
    "Log GDP per capita": "log_gdp_per_capita",
    "Social support": "social_support",
    "Healthy life expectancy at birth": "healthy_life_expectancy_at_birth",
    "Freedom to make life choices": "freedom_to_make_life_choices",
    "Generosity": "generosity",
    "Perceptions of corruption": "perceptions_of_corruption",
    "Positive affect": "positive_affect",
    "Negative affect": "negative_affect"
})
country_happiness_df.head()

In [ ]:
# Only show years 2015 - 2021
country_happiness_df = country_happiness_df.loc[(country_happiness_df['year'] >= 2015) & (country_happiness_df['year'] <= 2021)]
country_happiness_df.head()

In [ ]:
# Import the daily_weather_data csv

weather_df = pd.read_csv("./resources/daily_weather_data.csv")
weather_df

In [ ]:
# View column headers
weather_df.columns

In [ ]:
# Make all columns lowercase
weather_df = weather_df.rename(columns={
    "Latitude": "latitude",
    "Longitude": "longitude",
})
weather_df.head()

In [ ]:
# Converted date to proper YYYYMMDD
weather_df["date"] = pd.to_datetime(weather_df["date"], format='%d-%m-%Y')
weather_df.head()

In [ ]:
# Extract year component and create new column "year"
weather_df["year"] = weather_df["date"].dt.year

# Move "year" column next to "date" column
weather_df.insert(1, "year", weather_df.pop("year"))

weather_df.head()

In [ ]:
# Group data by year and country and calculate average temperature data
yearly_country_avg = weather_df.groupby(["year", "country"]).mean()
yearly_country_avg

In [ ]:
# Merge the Happiness and Weather dataframes together
merged_df = pd.merge(country_happiness_df, weather_df, on=["country", "year"])
merged_df.head()

In [ ]:
# Group merged_df by year and country
grouped_by_year_country = merged_df.groupby(["year", "country"])

# Calculate the mean of each group
yearly_country_mean = grouped_by_year_country.mean()
yearly_country_mean

In [ ]:
# Check for missing values in the dataframe
print(yearly_country_mean.isnull().sum())

In [ ]:
# Replace empty values with NaN
yearly_country_mean = yearly_country_mean.replace("", np.nan)

# Fill missing values with 0
yearly_country_mean = yearly_country_mean.fillna(0)

In [ ]:
# Check for missing values in the dataframe
print(yearly_country_mean.isnull().sum())

In [ ]:
# Check the data type
yearly_country_mean.info()

In [ ]:
from sqlalchemy import create_engine
from config import username, password, hostname, port, db

# username = "postgres"
# password = "postgres"
# hostname = "localhost"
# port = "5432"
# db = "project_3"

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{db}')


In [ ]:
# With enine.connect() as conn:
yearly_country_mean.to_sql("happiness_weather", con=engine, if_exists="replace", index=False)

In [ ]:
from sqlalchemy import text
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM happiness_weather"))
    for row in result:
        print(row)

In [ ]:
conn = engine.connect()
query = "SELECT * FROM happiness_weather"
df = pd.read_sql(query, conn)
print(df)

In [ ]:
# Export the csv file
# yearly_country_mean.to_csv("combined_happiness_weather.csv")

In [ ]:
import statsmodels.api as sm
# Load data from CSV file

data = merged_df

In [ ]:
# Create a linear regression model with life_ladder as the response variable and tavg as the predictor variable
model = sm.formula.ols('life_ladder ~ tavg', data=data).fit()
# Print the model summary
print(model.summary())

In [ ]:
# Load data from CSV file
data = merged_df
# Create a linear regression model with life_ladder as the response variable and tavg as the predictor variable
model = sm.formula.ols('life_ladder ~ pres', data=data).fit()
# Print the model summary
print(model.summary())

In [ ]:
# Load data from CSV file
data = merged_df
# Create a linear regression model with life_ladder as the response variable and tavg as the predictor variable
model = sm.formula.ols('life_ladder ~ wspd', data=data).fit()
# Print the model summary
print(model.summary())